# [**Reference**](https://www.kaggle.com/code/kaggle5daysofai/day-1b-agent-architectures)

In [31]:
import os
from kaggle_secrets import UserSecretsClient

GEMINI_API_KEY = UserSecretsClient().get_secret("GEMINI_API_KEY")
os.environ["GOOGLE_API_KEY"] = GEMINI_API_KEY

In [32]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

In [22]:
retry_config=types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504], 
)

In [6]:
#Research Agent
research_agent = Agent(
    name = 'ResearchAgent',
    model = Gemini(
        model = 'gemini-2.5-flash-lite',
        retry_options = retry_config
    ),
    instruction = """You are a specialized research agent. Your only job is to use the
    google_search tool to find 2-3 pieces of relevant information on the given topic and present the findings with citations.""",
    tools = [google_search],
    output_key = "research_findings"
)

In [7]:
#Summarizing Agent
summarizer_agent = Agent(
    name = 'SummarizerAgent',
    model = Gemini(
        model = 'gemini-2.5-flash-lite',
        retry_options = retry_config,
    ),
    instruction = """Read the provided research findings: {research_findings}
Create a concise summary as a bulleted list with 3-5 key points.""",
    output_key = "summary"
)

In [8]:
#Coordinator Agent
root_agent = Agent(
    name = 'RootAgent',
    model = Gemini(
        model = 'gemini-2.5-flash-lite',
        retry_options = retry_config,
    ),
    instruction = """You are a research coordinator. Your goal is to answer the user's query by orchestrating a workflow.
1. First, you MUST call the `ResearchAgent` tool to find relevant information on the topic provided by the user.
2. Next, after receiving the research findings, you MUST call the `SummarizerAgent` tool to create a concise summary.
3. Finally, present the summary clearly to the user as your response.""",
    tools = [AgentTool(research_agent), AgentTool(summarizer_agent)]
)

In [ ]:
# runner = InMemoryRunner(agent=root_agent)
# response = await runner.run_debug(
#     """..."""
# )

async with InMemoryRunner(agent=root_agent) as runner:
    response = await runner.run_debug(""""List some top resouces like blog, youtube channel, github true, hugging face, etc.. to learn Agentic AI, Data Science to learn and crack interviews""")

In [34]:
print(response)

[Event(model_version='gemini-2.5-flash-lite', content=Content(
  parts=[
    Part(
      function_call=FunctionCall(
        args={
          'request': 'Top resources for learning Agentic AI, Data Science, and interview preparation, including blogs, YouTube channels, GitHub repositories, and Hugging Face spaces.'
        },
        id='adk-1cfb7750-abb9-4e39-a0ca-54421b84797f',
        name='ResearchAgent'
      )
    ),
  ],
  role='model'
), grounding_metadata=None, partial=None, turn_complete=None, finish_reason=<FinishReason.STOP: 'STOP'>, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=GenerateContentResponseUsageMetadata(
  candidates_token_count=42,
  prompt_token_count=204,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=204
    ),
  ],
  total_token_count=246
), live_session_resumption_update=None, input_transcription=None, output_transcription=None, avg_logprobs=None,

In [37]:
print(response[1])

model_version=None content=Content(
  parts=[
    Part(
      function_response=FunctionResponse(
        id='adk-1cfb7750-abb9-4e39-a0ca-54421b84797f',
        name='ResearchAgent',
        response={
          'result': """Here are some top resources for learning Agentic AI, Data Science, and interview preparation:

## Agentic AI

**YouTube Channels:**
*   **Microsoft:** Offers a course on AI agents for beginners available on YouTube, covering fundamentals and providing code samples.
*   **Berkeley University:** Has a playlist of lectures on agentic AI and advanced LLM agents.
*   **Ed Donner:** Provides extensive teaching on LLMs and agentic AI through platforms like O'Reilly and Udemy, with a YouTube workshop available.
*   **Jon Krohn:** Co-founder of Y Carrot, a firm specializing in agentic AI systems, he also hosts SuperDataScience podcast and has a YouTube workshop.
*   **AI with Brandon:** Known for addressing business problems with agentic solutions.
*   **Cole Medin:** Focus

ERROR:asyncio:Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7e4acc508310>
ERROR:asyncio:Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7e4acc60bb10>
ERROR:asyncio:Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7e4acc5a1590>
ERROR:asyncio:Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7e4acc586f50>
ERROR:asyncio:Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7e4acc628d10>
ERROR:asyncio:Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7e4acc686b10>


In [9]:
outline_agent = Agent(
    name='OutlineAgent',
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Create a blog outline for the given topic with:
    1. A catchy headline
    2. An introduction hook
    3. 3-5 main sections with 2-3 bullet points for each
    4. A concluding thought""",
    output_key="blog_outline",
)

In [10]:
writer_agent = Agent(
    name="WriterAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    
    instruction="""Following this outline strictly: {blog_outline}
    Write a brief, 200 to 300-word blog post with an engaging and informative tone.""",
    output_key="blog_draft",
)

In [11]:
editor_agent = Agent(
    name="EditorAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),

    instruction="""Edit this draft: {blog_draft}
    Your task is to polish the text by fixing any grammatical errors, improving the flow and sentence structure, and enhancing overall clarity.""",
    output_key="final_blog",
)

In [12]:
root_agent = SequentialAgent(
    name="BlogPipeline",
    sub_agents = [outline_agent, writer_agent, editor_agent]
)

In [17]:
async with InMemoryRunner(agent=root_agent) as runner:
    response = await runner.run_debug("""write a blog to List some top resouces like blog, youtube channel, github true, hugging face, etc.. to learn Agentic AI, Data Science to learn and crack interviews""")


 ### Created new session: debug_session_id

User > write a blog to List some top resouces like blog, youtube channel, github true, hugging face, etc.. to learn Agentic AI, Data Science to learn and crack interviews
OutlineAgent > ## Outline: Your Launchpad to Agentic AI & Data Science Mastery (and Cracking Those Interviews!)

**Headline:** Beyond the Buzzwords: Your Ultimate Resource Guide to Mastering Agentic AI, Data Science, and Nailing Your Dream Job

**Introduction Hook:** Feeling overwhelmed by the explosion of Agentic AI and Data Science? Want to go from curious observer to job-ready expert? You're in the right place. This isn't just another list; it's your curated roadmap to the most impactful resources that will equip you with the knowledge, skills, and confidence to not only learn but also ace those competitive interviews.

---

**I. Building Your Foundational Pillars: The Essential Learning Hubs**

*   **The Power of Blogs & Articles:** Discover why staying updated with ind

In [19]:
print(response)

[Event(model_version='gemini-2.5-flash-lite', content=Content(
  parts=[
    Part(
      text="""## Outline: Your Launchpad to Agentic AI & Data Science Mastery (and Cracking Those Interviews!)

**Headline:** Beyond the Buzzwords: Your Ultimate Resource Guide to Mastering Agentic AI, Data Science, and Nailing Your Dream Job

**Introduction Hook:** Feeling overwhelmed by the explosion of Agentic AI and Data Science? Want to go from curious observer to job-ready expert? You're in the right place. This isn't just another list; it's your curated roadmap to the most impactful resources that will equip you with the knowledge, skills, and confidence to not only learn but also ace those competitive interviews.

---

**I. Building Your Foundational Pillars: The Essential Learning Hubs**

*   **The Power of Blogs & Articles:** Discover why staying updated with industry insights and practical tutorials is crucial.
    *   **Towards Data Science:** Your go-to for in-depth articles, tutorials, and 

In [23]:
initial_writing_agent = Agent(
    name="InitialWriterAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Based on the user's prompt, write the first draft of a short story (around 100-150 words).
    Output only the story text, with no introduction or explanation.""",
    output_key="current_story", 
) 

In [24]:
critic_agent = Agent(
    name="CriticAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a constructive story critic. Review the story provided below.
    Story: {current_story}
    
    Evaluate the story's plot, characters, and pacing.
    - If the story is well-written and complete, you MUST respond with the exact phrase: "APPROVED"
    - Otherwise, provide 2-3 specific, actionable suggestions for improvement.""",
    output_key="critique",
)

In [25]:
def exit_loop():
    """Call this function ONLY when the critique is 'APPROVED', indicating the story is finished and no more changes are needed."""
    return {"status": "approved", "message" : "Story approved. Exiting refinement loop."}

In [27]:
refiner_agent = Agent(
    name="RefinerAgent",
    model = Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a story refiner. You have a story draft and critique.
    
    Story Draft: {current_story}
    Critique: {critique}
    
    Your task is to analyze the critique.
    - IF the critique is EXACTLY "APPROVED", you MUST call the `exit_loop` function and nothing else.
    - OTHERWISE, rewrite the story draft to fully incorporate the feedback from the critique.""",
    output_key="current_story",
    tools=[FunctionTool(exit_loop)],
)

In [28]:
story_refinement_loop = LoopAgent(
    name="StoryRefinementLoop",
    sub_agents=[critic_agent, refiner_agent],
    max_iterations=4
)

In [29]:
root_agent = SequentialAgent(
    name="RootAgent",
    sub_agents = [initial_writing_agent, story_refinement_loop]
)

In [30]:
async with InMemoryRunner(agent=root_agent) as runner:
    response = await runner.run_debug("""write a story on Listing some top resouces like blog, youtube channel, github true, hugging face, etc.. to learn Agentic AI, Data Science to learn and crack interviews""")


 ### Created new session: debug_session_id

User > write a story on Listing some top resouces like blog, youtube channel, github true, hugging face, etc.. to learn Agentic AI, Data Science to learn and crack interviews
InitialWriterAgent > The flickering monitor cast a blue glow on Anya’s determined face. Interview season loomed, and the vast landscape of Agentic AI and Data Science felt like an uncharted ocean. She took a deep breath and opened a new tab.

First stop: Towards Data Science on Medium. Articles, tutorials, and real-world case studies – it was a treasure trove. Next, a quick jump to YouTube. I Am Deep Learning and Krish Naik’s channel were her go-to for clear explanations and coding walk-throughs.

For practical application, GitHub was essential. She bookmarked repositories like LangChain-ai and Auto-GPT, dissecting their code. Hugging Face was her playground for experimenting with pre-trained models, a crucial step for understanding AI in action.

Finally, a quick scan 

CriticAgent > APPROVED


CriticAgent > APPROVED
